# User CF + Item CF + SVD

In [1]:
import pandas as pd
import os

### Import data
The file "u.data" which contains the full dataset.
- 100,000 rating(1-5) from 943 users on 1682 movies
- Each user has rated at least 20 movies

In [2]:
data_dir = os.path.join('data', 'ml-100k')

In [3]:
data_dir

'data/ml-100k'

In [4]:
file_name = os.path.join(data_dir, 'u.data')

df = pd.read_csv(file_name, sep='\t', header = -1)

### full u dataset
- user id || item id || rating || timestape

Give the columns name to the dataset

In [5]:
df.columns = ['user_id', 'item_id', 'rating', 'itemstape']

In [6]:
df.head()

,user_id,item_id,rating,itemstape
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### Split dataset(train 20%, test 80%) 

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size = 0.2)

### Rebuild the matrix(n x m) which n is number of users and m is the number of movies

In [8]:
n_users = df.user_id.unique().shape[0]

n_items = df.item_id.unique().shape[0]

print('Number of users = ' + str(n_users) + '| number of movies = '+ str(n_items))

Number of users = 943| number of movies = 1682


### Create two user-item matrix, one for training and another for testing

### Training dataset

In [9]:
import numpy as np

train_data_matrix = np.zeros((n_users, n_items))

train_data_matrix.shape

(943, 1682)

### Testing dataset

In [10]:
test_data_matrix = np.zeros((n_users, n_items))

In [11]:
train_data.head()

,user_id,item_id,rating,itemstape
75154,521,550,3,885253844
21070,178,197,2,882826720
27306,62,91,4,879375196
39065,618,705,3,891307825
52703,643,200,3,891448265


### insert the data into the train data matrix and test data matrix

Becuase user_id and item_id are start from 1, so the value in the matrix should be decread by 1

In [12]:
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]

for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

### Data standarization

##### Get the mean of each row

In [13]:
train_rows, train_cols = train_data_matrix.nonzero()

X_row_mean = np.zeros(train_data_matrix.shape[0])
X_row_sum = np.zeros(train_data_matrix.shape[0])

for i in range(train_rows.shape[0]):
    X_row_mean[train_rows[i]] += train_data_matrix[train_rows[i], train_cols[i]]
    X_row_sum[train_rows[i]] += 1
    
X_row_mean /= X_row_sum + (X_row_sum == 0)

In [14]:
X_row_mean.shape

(943,)

##### Copy the data

In [15]:
train_matrix = train_data_matrix.copy()
test_matrix = test_data_matrix.copy()

X - $\overline{x}$

$\begin{bmatrix}
x_{1,1} && x_{1, 2} .... x_{1, n} \\
x_{2,1} && x_{2, 2} .... x_{2, n} \\
.\\
.\\
.\\
x{m, 1} && x_{m, 2} .... x_{m, n}
\end{bmatrix}
$
$
-
$
$
\begin{bmatrix}
\overline{x_{1}} \\
\overline{x_{2}} \\
.\\
.\\
.\\
\overline{x_{m}}
\end{bmatrix}
$

In [16]:
for i in range(train_rows.shape[0]):
    train_data_matrix[train_rows[i], train_cols[i]] -= X_row_mean[train_rows[i]]

In [17]:
train_data_matrix

array([[ 1.35321101, -0.64678899,  0.35321101, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.24      ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.8125    ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.62962963,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
test_rows, test_cols = test_data_matrix.nonzero()

for i in range(test_rows.shape[0]):
    test_data_matrix[test_rows[i], test_cols[i]] -= X_row_mean[test_rows[i]]


## Collaborative Flitering(Memory based)

There are two main parts of CF: User CF and Item CF. 

User CF: choose a specific user and based on the similarity of two users to recommend items(like: the person like these items also like that one)

Item CF: Find the persons who like the same item and these persons other like items. Calculate the similarity base on that


![title](data/User_base_and_item_base.jpeg)

### compute user and item similarity


In [19]:
import sklearn

##### The normal distance matrix in recommendation system we use the cosine similarity

User similarity:

$ S_{u}^{cosine}(U_{k},U_{a}) = \frac{U_{k}*U_{a}}{|U_{k}| |U_{a}|} = \frac{\sum{X_{k,m} * X_{a,m}}}{\sqrt{\sum{X_{k}^{2}}\sum{X^{2}_{a,m}}}} $

$ S_{u}^{cosine}(I_{m},I_{b}) = \frac{I_{m}*I_{b}}{|I_{m}| |I_{b}|} = \frac{\sum{X_{a,m} * X_{a,b}}}{\sqrt{\sum{X_{a,m}^{2}}\sum{X^{2}_{a,b}}}} $

In [20]:
user_similarity = sklearn.metrics.pairwise_distances(train_data_matrix, metric = "cosine")

item_similarity = sklearn.metrics.pairwise_distances(train_data_matrix.T, metric = "cosine")

#### The shape of the user similarity matrix becomes 943 x 943 

In [21]:
user_similarity.shape

(943, 943)

#### The shape of the item similarity matrix becomes 1682 x 1682

In [22]:
item_similarity.shape

(1682, 1682)

### Prediction based on either user or items

Prediction function is based on:
    $x_{k,m} = \overline{x_{k}} + \frac{\sum_{u_{a}}{sim_{u}(u_{k}, u_{a})(x_{a,m} - \overline{x_{u_{a}}})}}{\sum_{u_{a}}|sim_{u}(u_{k}, u_{a})|}$

<img src = "data/user_base.png" width = 300>

##### The intuition here is that not everyone rating standard is same. 
##### For example, if user A like a movie he might give the movie 5 stars and others 3 or 4. But some picky people rarely give 5 star to the movie. 
##### So the algorithm use the similarity as the weight to predict rating

In [23]:
len(train_data_matrix)

943

In [24]:
def c_mean(mat):
    r = []
    for i in range(len(mat)):
        num = 0
        a = 0
        for n in mat[i]:
            if n != 0:
                a = a + n
                num += 1
        r.append(a/num)
    return r

In [25]:
def predict(ratings, similarity, type='user'):
        if type == 'user':
            mean_user_rating = np.array(c_mean(ratings))
            #You use np.newaxis so that mean_user_rating has same format as ratings
            ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
            pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
        elif type == 'item':
            pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
        return pred

In [26]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')
# user_prediction = predict(train_data_matrix, item_similarity, type='user')

### Evaluation

##### The most popular way to evaluate the model is using RMSE(root-mean-square error)

$RMSE = \sqrt{\frac{1}{N}\sum(x_{i} - \hat{x_{i}})^{2}}$

In [27]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [28]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [29]:
print ('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 1.0247059555804348
Item-based CF RMSE: 1.044138632710753


#### Conclusion:
Memory-based CF is easy to implement but the real problem is cold start, which means when the new users or data come

### Collabrative flitering base on SVD

##### Calculate the sparse level of the matrix

In [30]:
sparsity = round(1.0 - len(df)/float(n_users*n_items),3)

print('The sparsity level of the MovieLens 100K is ' + str(sparsity * 100) + '%')

The sparsity level of the MovieLens 100K is 93.7%


#### Use scipy svd

In [31]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

In [32]:
U, s, Vt = svds(train_data_matrix, k = 10)

In [33]:
pred = U.dot(np.diag(s)).dot(Vt)

In [34]:
pred.shape

(943, 1682)

In [35]:
print(rmse(pred, test_data_matrix))

0.9853430688879861


In [36]:
train_data_matrix.shape

(943, 1682)

## Some Math notes

### SVD Algorithm

In linear algebra, the singular-value decomposition is a factorization of a real and complex matrix. 

##### Intuition of the SVD: Each operation of the matrix is rotation and streching

### Example

X = $\begin{bmatrix}
1\\
3
\end{bmatrix}
$
and 
$
A = \begin{bmatrix}
2 & 1\\
-1 & 1
\end{bmatrix}
$

$
Y = Ax = 
\begin{bmatrix}
2 & 1\\
-1 & 1
\end{bmatrix}
$
$
\begin{bmatrix}
1 \\
3
\end{bmatrix}
$
= $
\begin{bmatrix}
5 \\
2
\end{bmatrix}
$

##### rotation function

$A = \begin{bmatrix}
cos\Theta & -sin\Theta\\
sin\Theta & cos\Theta
\end{bmatrix}
$
which is rotated by $\Theta$

##### Streching function

$A = \begin{bmatrix}
\alpha & 0\\
0 & \alpha
\end{bmatrix}$
which $\alpha$ = "streching"

##### General function:

$AV = U\sum$
which $u$ is the unit matrix and $\sum$ is streching matrix

It is like a enignvalues function which is :
$A\overline{x} = \lambda\overline{x}$

When both sides of the equation multiply by $V^{-1}$

$A = U\sum V^{-1}$

And we let $V^{-1} = V^{*}$

$A = U\sum V^{*}$ which is the SVD function

#### Solve this function

$A^{T}A = (U\sum V^{*})^{T}(U\sum V^{*})$

$= (V\sum U^{*})(U\sum V^{*})$

$=V\sum^{2}V^{*}$

##### Final equation:
$A^{T}AV = V\sum^{2}V^{*}V$

$A^{T}AV = V\sum^{2}$ which is the enigvalue problem:

$A^{T}A = A$


$\sum^{2}$ is $\lambda$
##### enignvector is: V

#### To get U use the same way:
The both side of equation right multyplied by $A^{T}$<br>
which is $AA^{T} = (U\sum V^{*})(U\sum V^{*})^{T}$

$AA^{T}U = U\sum^{2}$

### Solve eignvalues and eignvector problem

##### Defination:
Let $A$ be an $n*n$ matrix. A scalar $lambda$ is called an eigenvalue of $A$ if there is a nonzero vector $\overline{x}$ such that $A\overline{x} = \lambda\overline{x}$. So sunch a vector $\overline{x}$ is called an eigenvector of $A$ corresponding to $\lambda$

### Example

Show that $\overline{x} = \begin{bmatrix}
3 & 2\\
3 & -2
\end{bmatrix}
$
corresponding to 
$\lambda$ = 4

##### note:
If $\lambda$ is an eigenvalue of A, and $\overline{x}$ is an eigenvector belonging to $\lambda$,<br>
any nonzero multiple of $\overline{x}$ will be an eigenvector

### Finding Eigenvalues and Eigenvector

#### 2x2 matrix example 

The process: <br>
To solve for the eigenvalues, $\lambda_{i}$ and the corresponding eigenvectors, $\overline{x_{i}}$, of an $n x n$ matrix $A$, do the following:<br>
- Multiply an $n x n$ identity matrix by the scalar
- subtract the identity matrix multiple from the matrix A
- Find the determined of the matrix and the difference
- solve for the values of $\lambda$ that satisfy the equation: $det(A - \lambda I) = \overline{0}$
- Solve for the corresponding vector to each $\lambda$

#### find the eigenvalues and eigenvectors of the matrix

$A = \begin{bmatrix}
7 & 3\\
3 & -1
\end{bmatrix}$

1. $\lambda I = \lambda \begin{bmatrix}
1 & 0\\
0 & 1
\end{bmatrix}
$ = 
$
\begin{bmatrix}
\lambda & 0\\
0 & \lambda
\end{bmatrix}
$

2.  $A - \lambda I = \begin{bmatrix}
7-\lambda & 3\\
3 & -1-\lambda
\end{bmatrix}
$

Solve this equation we can get a $\lambda$ = 8 which is the eigenvalue

To find the corresponding eigenvector: <br>
$A - \lambda I$ = B<br>
And let $B\overline{x} = \overline{0}$

Solve the equation to get the eigenvector $\overline{x}$

## Use gradient descent on SVD

##### The lose function: 

<img src="data/min_function.png" width = 300>

#### paramters update equations:

<img src = "data/parameters_update.png" width = 300>

In [44]:
def svd(mat, k, steps = 100, gama = 0.02, lamda = 0.3):
    slow_rate = 0.99
    pre_rmse = 100000
    now_rmse = 0
    
    user_feature = np.matrix(np.random.rand(mat.shape[0], k))
    item_feature = np.matrix(np.random.rand(mat.shape[1], k))
    
    for step in range(steps):
        rmse= 0
        n = 0
        
        for u in range(mat.shape[0]):
            for i in range(mat.shape[1]):
                if mat[u, i] != 0:
                    pui = float(np.dot(user_feature[u, :], item_feature[i, :].T))
                    eui = mat[u, i] - pui
                    rmse += pow(eui, 2)
                    n += 1
                    
                    for feature in range(k):
                        user_feature[u, feature] += gama*(eui*item_feature[i, feature] - lamda * user_feature[u, feature])
                        item_feature[i, feature] += gama*(eui*user_feature[u, feature] - lamda*item_feature[i, feature])
        now_rmse = sqrt(rmse * 1 / n)
            
        print('step: %d     rmse:%s'%((step+1), now_rmse))
        
        if(now_rmse < pre_rmse):
            pre_rmse = now_rmse
        else:
            return (user_feature, item_feature)
        
        gama *= slow_rate
        step += 1
    return (user_feature, item_feature)

In [45]:
U, V = svd(train_matrix, k = 10)

step: 1     rmse:1.0746054840015045
step: 2     rmse:0.9968375025264921
step: 3     rmse:0.9933076710484064
step: 4     rmse:0.9918369122138417
step: 5     rmse:0.9910924755717536
step: 6     rmse:0.9906115740794252
step: 7     rmse:0.9902467787167445
step: 8     rmse:0.989938551750606
step: 9     rmse:0.9896593852949608
step: 10     rmse:0.9893953907131786
step: 11     rmse:0.9891390821576653
step: 12     rmse:0.9888862025848304
step: 13     rmse:0.9886342079894223
step: 14     rmse:0.9883814971634322
step: 15     rmse:0.9881270009136445
step: 16     rmse:0.9878699545364731
step: 17     rmse:0.9876097681255973
step: 18     rmse:0.9873459512005311
step: 19     rmse:0.9870780685633902
step: 20     rmse:0.9868057146973354
step: 21     rmse:0.9865284995309523
step: 22     rmse:0.9862460414051553
step: 23     rmse:0.9859579647709095
step: 24     rmse:0.9856639011136511
step: 25     rmse:0.9853634921604842
step: 26     rmse:0.9850563947501898
step: 27     rmse:0.9847422869294049
step: 28   

In [46]:
pred = np.array(np.dot(U, V.T))

In [47]:
print("rmse on test data matrix: " + str(rmse(pred, test_matrix)))

rmse on test data matrix: 1.0078240348988312
